In [7]:
import cPickle as pickle
import pandas as pd, numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [8]:
train = pd.read_csv('./final/train_800.csv')
test = pd.read_csv('./test.csv')
subm = pd.read_csv('./sample_submission.csv')



In [9]:
test.drop(test.index, inplace=True)

In [10]:
test.head()

,id,comment_text


In [11]:
train.count()

Unnamed: 0       868
comment_text     868
id               868
identity_hate    868
insult           868
obscene          868
severe_toxic     868
threat           868
toxic            868
dtype: int64

In [12]:
test = pd.DataFrame([['1', 'fuck hisham'],
                     ['2', 'Ill kill her'],
                     ['3', 'I hate her'],
                     ['4', 'suck my dick'],
                     ['5','My teacher hates me, never listens to my reasoning'],
                     ['6','I feel like murdering her'],
                     ['7','hey dont say that'],
                     ['8','what am I gonna do about her?'],
                     ['9','idk'],
                     ['10','I\'ll probably murder her']
                    ], columns=['id', 'comment_text'])

In [13]:
test.head()

,id,comment_text
0,1,fuck hisham
1,2,Ill kill her
2,3,I hate her
3,4,suck my dick
4,5,"My teacher hates me, never listens to my reaso..."


In [14]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
train['none'] = 1-train[label_cols].max(axis=1)
train.describe()

,Unnamed: 0,identity_hate,insult,obscene,severe_toxic,threat,toxic,none
count,868.000000,868.000000,868.000000,868.000000,868.000000,868.000000,868.000000,868.000000
mean,433.500000,0.138249,0.322581,0.271889,0.112903,0.157834,0.599078,0.358295
std,250.714313,0.345360,0.467733,0.445190,0.316657,0.364796,0.490368,0.479776
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,216.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,433.500000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,650.250000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000
max,867.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [15]:
len(train),len(test)

(868, 10)

In [16]:
COMMENT = 'comment_text'
train[COMMENT].fillna("unknown", inplace=True)
test[COMMENT].fillna("unknown", inplace=True)

In [17]:
import re,string
re_tok = re.compile(re.escape(string.punctuation))

In [18]:
def tokenize(s): return re_tok.sub(r' \1 ', s).split()

In [19]:
n = train.shape[0]
vec = TfidfVectorizer(ngram_range=(1,2), tokenizer=tokenize,
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1 )
trn_term_doc = vec.fit_transform(train[COMMENT])
with open('/home/ammar/study/8th semester/FYP/toxic comments/final/vec','wb') as fp:
        pickle.dump(vec,fp)
test_term_doc = vec.transform(test[COMMENT])

In [20]:
def pr(y_i, y):
    p = x[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum()+1)
x = trn_term_doc
test_x = test_term_doc

In [21]:
def get_mdl(y):
    y = y.values
    r = np.log(pr(1,y) / pr(0,y))
    m = LogisticRegression(C=4, dual=True)
    models = {}
    x_nb = x.multiply(r)
    result = m.fit(x_nb, y)
    return result, r,m
get_mdl.counter = 0

In [22]:
preds = np.zeros((len(test), len(label_cols)))
models = {}
# print(test_x)

In [24]:
for i, j in enumerate(label_cols):
    print('fit', j)
    m,r,model = get_mdl(train[j])
    print r
    with open('/home/ammar/study/8th semester/FYP/toxic comments/final/model'+str(i),'wb') as fp:
        pickle.dump(m,fp)
    with open('/home/ammar/study/8th semester/FYP/toxic comments/final/r'+str(i),'wb') as fp:
        pickle.dump(r,fp)
    preds[:,i] = m.predict_proba(test_x.multiply(r))[:,1]

('fit', 'toxic')
[[-0.79076703 -0.74313494 -0.3347077  ... -0.62140345 -0.62140345
  -0.45970647]]
('fit', 'severe_toxic')
[[-0.47872741  1.49067633  1.48148895 ...  1.50789906  1.50789906
   1.53884039]]
('fit', 'obscene')
[[-0.30218424  0.63995346  0.74995376 ...  0.76168495  0.76168495
   0.66811577]]
('fit', 'threat')
[[-0.90280662  1.10663464  1.09744725 ...  1.12385737  1.12385737
   1.1547987 ]]
('fit', 'insult')
[[-0.77734082  0.17817932  0.51298344 ...  0.19540205  0.19540205
   0.22634338]]
('fit', 'identity_hate')
[[-0.51266491  1.35803089  1.07250258 ...  1.60222311  1.60222311
   1.3092203 ]]


In [25]:
print r.shape
print m
print test_x.shape

(1, 5172)
LogisticRegression(C=4, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
(10, 5172)


In [26]:
submid = pd.DataFrame({'id': test["id"]})
submission = pd.concat([submid, pd.DataFrame(preds, columns = label_cols)], axis=1)
# submission.to_csv('submission.csv', index=False)

In [27]:
train.count()

Unnamed: 0       868
comment_text     868
id               868
identity_hate    868
insult           868
obscene          868
severe_toxic     868
threat           868
toxic            868
none             868
dtype: int64

In [28]:
print(models)

{}


In [29]:
test

,id,comment_text
0,1,fuck hisham
1,2,Ill kill her
2,3,I hate her
3,4,suck my dick
4,5,"My teacher hates me, never listens to my reaso..."
5,6,I feel like murdering her
6,7,hey dont say that
7,8,what am I gonna do about her?
8,9,idk
9,10,I'll probably murder her


In [30]:
submission

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,1,0.997419,0.120052,0.999995,0.070855,0.997788,0.062821
1,2,0.993382,0.038631,0.106894,0.788368,0.165718,0.084416
2,3,0.934991,0.047457,0.072043,0.217949,0.887544,0.576821
3,4,0.674076,0.031495,0.314479,0.041746,0.392381,0.039716
4,5,0.644508,0.026411,0.151073,0.088909,0.366366,0.138414
5,6,0.630234,0.077814,0.269465,0.058059,0.393105,0.155269
6,7,0.665528,0.058804,0.375469,0.033629,0.242779,0.057280
7,8,0.686601,0.027260,0.068855,0.067279,0.137973,0.031747
8,9,0.619981,0.031644,0.065670,0.046189,0.143299,0.040259
9,10,0.945803,0.024166,0.258371,0.459792,0.324812,0.101500


In [25]:
len(submission)

NameError: name 'submission' is not defined

In [222]:
train = pd.read_csv('./train.csv')

In [223]:
train.tail()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0
159570,fff46fc426af1f9a,"""\nAnd ... I really don't think you understand...",0,0,0,0,0,0


In [224]:
train_plain =  train[(train['toxic'] == 0) & (train['severe_toxic'] == 0) & (train['obscene'] == 0) & (train['threat'] == 0) 
                     & (train['insult'] == 0) & (train['identity_hate'] == 0)]

In [225]:
train_plain.count()

id               143346
comment_text     143346
toxic            143346
severe_toxic     143346
obscene          143346
threat           143346
insult           143346
identity_hate    143346
dtype: int64

In [233]:
murder = train_plain[train_plain['comment_text'].apply(lambda x: 'murder' in x)]
shoot = train_plain[train_plain['comment_text'].apply(lambda x: 'shoot' in x)]
kill = train_plain[train_plain['comment_text'].apply(lambda x: 'kill' in x)]
bomb = train_plain[train_plain['comment_text'].apply(lambda x: 'bomb' in x)]

In [235]:
print murder.count()
print shoot.count()
print kill.count()
print bomb.count()

id               494
comment_text     494
toxic            494
severe_toxic     494
obscene          494
threat           494
insult           494
identity_hate    494
dtype: int64
id               322
comment_text     322
toxic            322
severe_toxic     322
obscene          322
threat           322
insult           322
identity_hate    322
dtype: int64
id               1476
comment_text     1476
toxic            1476
severe_toxic     1476
obscene          1476
threat           1476
insult           1476
identity_hate    1476
dtype: int64
id               335
comment_text     335
toxic            335
severe_toxic     335
obscene          335
threat           335
insult           335
identity_hate    335
dtype: int64


In [ ]:
murder_50 = murder.sample(n=50)
murder_50.count()

In [135]:
sample_300 = train_plain.sample(n=300)
sample_300.count()

id               300
comment_text     300
toxic            300
severe_toxic     300
obscene          300
threat           300
insult           300
identity_hate    300
dtype: int64

In [140]:
toxic_plain = pd.concat([train1,sample_300])
toxic_plain_shuffled = toxic_plain.sample(frac =1).reset_index(drop = True)
toxic_plain_shuffled.count()

id               408
comment_text     408
toxic            408
severe_toxic     408
obscene          408
threat           408
insult           408
identity_hate    408
dtype: int64

In [142]:
toxic_plain_shuffled.to_csv('/home/ammar/study/8th semester/FYP/toxic comments/train_created1')